In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?![\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"([, Alice, 's, Adventures, in, Wonderland, by,...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,alteration,merely,rest,inch,occasional,eager,think,like,son,sluggard,...,toast,enquire,defy,renew,graze,bother,conference,discredit,text_sentence,text_source
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"([, Alice, 's, Adventures, in, Wonderland, by,...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9871513632090254

Test set score: 0.8928571428571429


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3191, 3065) (3191,)
Training set score: 0.9561266060795989

Test set score: 0.9158834586466166


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8878094641178313

Test set score: 0.8707706766917294


In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

[Emma by Jane Austen 1816] Emma Woodhouse, handsome, clever, and rich, with a comfortable home and h


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [15]:
# Now we can model it!
# Let's use logistic regression again.
from sklearn.model_selection import cross_val_score

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
                              emma_bow.drop(['text_sentence','text_source'], 1)),
                             axis=0)

y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nEmma test set score:', cross_val_score(lr, X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Emma test set score: [0.76759777 0.7360179  0.77516779]


col_0,Austen,Carroll
row_0,,
Austen,1556,113
Carroll,706,308


In [16]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
train = knn.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', train.score(X_train, y_train))
print('\nTest set score:', train.score(X_test, y_test))
print('\nEmma test set score:', cross_val_score(knn, X_Emma_test, y_Emma_test))

(3191, 3065) (3191,)
Training set score: 0.8931369476653087

Test set score: 0.7777255639097744

Emma test set score: [0.63351955 0.62863535 0.58277405]


In [18]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
train = svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
print('\nEmma test set score:', cross_val_score(svc, X_Emma_test, y_Emma_test))

Training set score: 0.9815104982764024

Test set score: 0.9060150375939849

Emma test set score: [0.74078212 0.71029083 0.75727069]


## Make new features

In [19]:
counts = []
for x in sentences[0]:
    counts.append(len(x))

In [20]:
sentences['word_count'] = counts

In [24]:
alice_pos = []
for token in alice_doc:
    alice_pos.append(token.pos_)

In [26]:
# copy their feature maker for your own
def POS_features(sentences, speech_parts):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=speech_parts)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, speech_parts] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        parts = [token.pos_
                 for token in sentence
                 if (token.pos_ in speech_parts)]
        
        # Populate the row with word counts.
        for part in parts:
            df.loc[i, part] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [25]:
alice_speech_parts = pd.unique(alice_pos)
alice_speech_parts

array(['PUNCT', 'PROPN', 'PART', 'ADP', 'NUM', 'VERB', 'ADV', 'ADJ',
       'NOUN', 'DET', 'CCONJ', 'PRON', 'INTJ'], dtype=object)

In [104]:
persuasion_pos = []
for token in persuasion_doc:
    persuasion_pos.append(token.pos_)

persuasion_speech_parts = pd.unique(persuasion_pos)
persuasion_speech_parts

array(['PUNCT', 'NOUN', 'ADP', 'PROPN', 'NUM', 'VERB', 'DET', 'ADJ',
       'ADV', 'PART', 'CCONJ', 'PRON', 'INTJ', 'X'], dtype=object)

In [106]:
#speech_parts = set(alice_speech_parts + persuasion_speech_parts)

pos_df = POS_features(sentences, alice_speech_parts)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


In [29]:
pos_df = pd.concat([persuasion_pos_df, alice_pos_df], axis=1, ignore_index=True)

In [107]:
pos_df

,PUNCT,PROPN,PART,ADP,NUM,VERB,ADV,ADJ,NOUN,DET,CCONJ,PRON,INTJ,text_sentence,text_source
0,12,7,3,10,1,13,3,3,12,5,6,3,0,"([, Alice, 's, Adventures, in, Wonderland, by,...",Carroll
1,7,2,1,8,0,11,7,7,8,6,2,4,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,4,2,1,4,0,5,6,1,2,3,1,2,2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,1,0,0,0,0,0,0,0,0,0,0,0,2,"(Oh, dear, !)",Carroll
4,2,0,0,0,0,2,0,1,0,0,0,1,0,"(I, shall, be, late, !, ')",Carroll
5,17,4,6,19,0,19,10,5,14,11,8,13,0,"((, when, she, thought, it, over, afterwards, ...",Carroll
6,2,1,2,3,0,4,5,0,2,2,0,2,0,"(In, another, moment, down, went, Alice, after...",Carroll
7,4,1,1,5,0,7,11,1,5,5,1,3,0,"(The, rabbit, -, hole, went, straight, on, lik...",Carroll
8,3,0,4,4,0,9,4,1,4,1,3,4,0,"(Either, the, well, was, very, deep, ,, or, sh...",Carroll
9,6,0,4,5,0,11,4,1,7,2,4,5,0,"(First, ,, she, tried, to, look, down, and, ma...",Carroll


In [108]:
# Parts Of Speech model here!
Y = pos_df['text_source']
X = pos_df.drop(['text_sentence', 'text_source'], 1)

pos_x_train, pos_x_test, pos_y_train, pos_y_test = train_test_split(X, Y, test_size=0.4)

In [75]:
pos_y_train = pos_y_train=='Austen'

In [110]:
# try a bunch of models
# whee

train = svc.fit(pos_x_train, pos_y_train)
train = knn.fit(pos_x_train, pos_y_train)
train = lr.fit(pos_x_train, pos_y_train)

print('\nLogistic Regression Train set score:', cross_val_score(lr, pos_x_train, pos_y_train))
print('\nTest set score:', cross_val_score(lr, pos_x_test, pos_y_test))
print('\nSVC Train set score:', cross_val_score(svc, pos_x_train, pos_y_train))
print('\nTest set score:', cross_val_score(svc, pos_x_test, pos_y_test))
print('\nKNN Train set score:', cross_val_score(knn, pos_x_train, pos_y_train))
print('\nTest set score:', cross_val_score(knn, pos_x_test, pos_y_test))




Logistic Regression Train set score: [0.75774648 0.73095014 0.74788335]

Test set score: [0.76478873 0.73201693 0.74047955]

SVC Train set score: [0.75868545 0.73377234 0.73659454]

Test set score: [0.76197183 0.73201693 0.74471086]

KNN Train set score: [0.7342723 0.7130762 0.7215428]

Test set score: [0.71267606 0.73201693 0.71650212]


# checking model against new text

In [32]:
burgess = gutenberg.raw('burgess-busterbrown.txt')
burgess = re.sub('\r\n', ' ', burgess)
#burgess = re.sub(r'CHAPTER \w+', '', burgess)
#burgess = text_cleaner(burgess)



In [34]:
# Parse our cleaned data.
burgess_doc = nlp(burgess)

In [36]:
# Group into sentences.
burgess_sents = [[burgess_doc, "Burgess"] for sent in burgess_doc.sents]

# make sure everything is the same size as alice
burgess_sents = burgess_sents[0:len(alice_sents)].copy()

In [37]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
burgess_sentences = pd.DataFrame(burgess_sents)
burgess_bow = bow_features(burgess_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [38]:
X_burgess_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
                              burgess_bow.drop(['text_sentence','text_source'], 1)),
                             axis=0)
y_burgess_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Burgess'] * burgess_bow.shape[0])])

# Model.
print('\nTest set score:', cross_val_score(lr, X_burgess_test, y_burgess_test))
lr_burgess_predicted = lr.predict(X_burgess_test)
pd.crosstab(y_burgess_test, lr_burgess_predicted)


Test set score: [1. 1. 1.]


col_0,Austen,Carroll
row_0,,
Burgess,0,1050
Carroll,706,308
